In [ ]:
from github import GitHubManager
from google.colab import userdata


# Starte den GitHub-Sync
git_manager = GitHubManager(
    userdata.get("github_pat"),
    userdata.get("github_email"),
    userdata.get("github_username"),
    userdata.get("github_repo_seo")
)

git_manager.clone_repo()  # Klonen des Repos
git_manager.sync_project()  # Projekt in GitHub hochladen